# Format 3000,5000,5000 exclusive data to word lists
format 3000,5000,5000_exclusve in the following formats:

* all data text british: word, type, definition, example, phonetics, cefr
* all data text usa: word, type, definition, example, phonetics, cefr
* all data + pronunciation: same as above with clickable HTML
* 2 column: word, type, definiton

All of the above grouped by cefr

In [17]:
import pandas as pd
import os 
import re

In [18]:
DATASET = 'oxford_3000'
#DATASET = 'oxford_5000'
#DATASET = 'oxford_5000_exclusive'
df = pd.read_pickle(f"./data/{DATASET}.pkl")
df.head()

,word,type,cefr,phon_br,phon_n_am,definition,example,uk,us
0,a,indefinite article,a1,/ə/,/ə/,used before countable or singular nouns referr...,a man/horse/unit,a_uk.mp3,a_us.mp3
1,abandon,verb,b2,/əˈbændən/,/əˈbændən/,"to leave somebody, especially somebody you are...","abandon somebody, The baby had been abandoned ...",abandon_uk.mp3,abandon_us.mp3
2,ability,noun,a2,/əˈbɪləti/,/əˈbɪləti/,the fact that somebody/something is able to do...,People with the disease may lose their ability...,ability_uk.mp3,ability_us.mp3
3,able,adjective,a2,/ˈeɪbl/,/ˈeɪbl/,"to have the skill, intelligence, opportunity, ...",You must be able to speak French for this job.,able_uk.mp3,able_us.mp3
4,abolish,verb,c1,/əˈbɒlɪʃ/,/əˈbɑːlɪʃ/,"to officially end a law, a system or an instit...",This tax should be abolished.,abolish_uk.mp3,abolish_us.mp3


## HTML+PDF all columns alphabetical

In [19]:
# Complete to HTML
def replace_word_in_example_with_underscore(word, example):
    example_split = example.split(' ')
    def _replace(e):
        if word not in e:
           return e 
        if not re.match(f"^{word}.*?$", e):
            return e
        return e.replace(word, '_')
    example_split_replaced = list(map(lambda e: _replace(e), example_split))
    return ' '.join(example_split_replaced)

def load_data():
    data = df[["word", "type", "cefr", "phon_br", "phon_n_am", "definition", "example"]]
    data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())
    data = data.rename(columns={'phon_br' : 'phonetics (UK)'})
    data = data.rename(columns={'phon_n_am' : 'phonetics (US)'})
    return data

data = load_data()

style = data.style.format(
    escape="html",
    )
style = style.hide(axis='index')

html = style.to_html()
filename = DATASET + '_alphabetical'
with open(f'output/{filename}.html', 'w') as f:
    f.write(html)

cmd = f'pandoc -f html -t pdf output/{filename}.html -t html5 -o output/{filename}.pdf --metadata pagetitle="{filename}" -V margin-top=2 -V margin-bottom=2 -V margin-left=2 -V margin-right=2 -c format/table.css --pdf-engine-opt=--enable-local-file-access'
os.system(cmd)

/tmp/ipykernel_119920/3163655802.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())
Loading pages (1/6)


In [ ]:
# Complete to HTML
data = load_data()
data["example"] = data.apply(lambda row: replace_word_in_example_with_underscore(row.word, row.example) , axis=1)

style = data.style.format(
    escape="html",
    )
style = style.hide(axis='index')

html = style.to_html()
filename = DATASET + '_underscore_alphabetical'
with open(f'output/{filename}.html', 'w') as f:
    f.write(html)

cmd = f'pandoc -f html -t pdf output/{filename}.html -t html5 -o output/{filename}.pdf --metadata pagetitle="{filename}" -V margin-top=2 -V margin-bottom=2 -V margin-left=2 -V margin-right=2 -c format/table.css --pdf-engine-opt=--enable-local-file-access'
os.system(cmd)

/tmp/ipykernel_119920/3163655802.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                          


0

## HTML+PDF all columns grouped by CEFR

In [ ]:
data = load_data()
data["example"] = data.apply(lambda row: replace_word_in_example_with_underscore(row.word, row.example) , axis=1)
print(data["example"][0])
cefrs = ['A1', 'A2', 'B1', 'B2', 'C1']
data_by_cefr = list(map(lambda c : data[data['cefr'] == c], cefrs))

# Complete to HTML
html_out = ''
for data in data_by_cefr:
    if data.empty:
        continue
    cefr = data['cefr'].iloc[0]
    html_out += f'<h2>{cefr}</h2>'
    data = data.drop(['cefr'], axis=1)
    print()
    data = data.rename(columns={'word' : f'word ({cefr})'})

    style = data.style.format(
        escape="html",
        )
    style = style.hide(axis='index')
    html_out += style.to_html()


filename = DATASET+'_underscore_by_cefr'
with open(f'output/{filename}.html', 'w', encoding='utf-8') as f:
    f.write(html_out)

# to pdf
cmd = f"""pandoc -f html -t pdf output/{filename}.html -t html5 -o output/{filename}.pdf --metadata pagetitle="{filename}" -V margin-top=2 -V margin-bottom=2 -V margin-left=2 -V margin-right=2 -c format/table.css --pdf-engine-opt=--enable-local-file-access --title '{filename}'"""
os.system(cmd)


/tmp/ipykernel_119920/3163655802.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


This tax should be _ed.




Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                          


0

In [ ]:
data = load_data()
cefrs = ['A1', 'A2', 'B1', 'B2', 'C1']
data_by_cefr = list(map(lambda c : data[data['cefr'] == c], cefrs))

# Complete to HTML
html_out = ''
for data in data_by_cefr:
    if data.empty:
        continue
    cefr = data['cefr'].iloc[0]
    html_out += f'<h2>{cefr}</h2>'
    data = data.drop(['cefr'], axis=1)
    print()
    data = data.rename(columns={'word' : f'word ({cefr})'})

    style = data.style.format(
        escape="html",
        )
    style = style.hide(axis='index')
    html_out += style.to_html()


filename = DATASET+'_by_cefr'
with open(f'output/{filename}.html', 'w', encoding='utf-8') as f:
    f.write(html_out)

# to pdf
cmd = f"""pandoc -f html -t pdf output/{filename}.html -t html5 -o output/{filename}.pdf --metadata pagetitle="{filename}" -V margin-top=2 -V margin-bottom=2 -V margin-left=2 -V margin-right=2 -c format/table.css --pdf-engine-opt=--enable-local-file-access --title '{filename}'"""
os.system(cmd)


/tmp/ipykernel_119920/3163655802.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                          


0

## HTML+PDF all columns grouped by CEFR shuffle

In [ ]:
# Complete to HTML
html_out = ''
for data in data_by_cefr:
    if data.empty:
        continue
    data = load_data()
    cefr = data['cefr'].iloc[0]
    html_out += f'<h2>{cefr}</h2>'
    data = data.drop(['cefr'], axis=1)
    print()
    data = data.rename(columns={'word' : f'word ({cefr})'})
    data = data.sample(frac=1)

    style = data.style.format(
        escape="html",
        )
    style = style.hide(axis='index')
    html_out += style.to_html()


filename = DATASET+'_by_cefr_shuffle'
with open(f'output/{filename}.html', 'w', encoding='utf-8') as f:
    f.write(html_out)

# to pdf
cmd = f"""pandoc -f html -t pdf output/{filename}.html -t html5 -o output/{filename}.pdf --metadata pagetitle="{filename}" -V margin-top=2 -V margin-bottom=2 -V margin-left=2 -V margin-right=2 -c format/table.css --pdf-engine-opt=--enable-local-file-access --title '{filename}'"""
os.system(cmd)


/tmp/ipykernel_119920/3163655802.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())
/tmp/ipykernel_119920/3163655802.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                          


0

In [ ]:
# Complete to HTML
html_out = ''
for data in data_by_cefr:
    if data.empty:
        continue
    data = load_data()
    cefr = data['cefr'].iloc[0]
    html_out += f'<h2>{cefr}</h2>'
    data = data.drop(['cefr'], axis=1)
    print()
    data = data.rename(columns={'word' : f'word ({cefr})'})
    data = data.sample(frac=1)

    style = data.style.format(
        escape="html",
        )
    style = style.hide(axis='index')
    html_out += style.to_html()


filename = DATASET+'_by_cefr_shuffle'
with open(f'output/{filename}.html', 'w', encoding='utf-8') as f:
    f.write(html_out)

# to pdf
cmd = f"""pandoc -f html -t pdf output/{filename}.html -t html5 -o output/{filename}.pdf --metadata pagetitle="{filename}" -V margin-top=2 -V margin-bottom=2 -V margin-left=2 -V margin-right=2 -c format/table.css --pdf-engine-opt=--enable-local-file-access --title '{filename}'"""
os.system(cmd)


/tmp/ipykernel_119920/3163655802.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())
/tmp/ipykernel_119920/3163655802.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                          


0

## 2 Column LateX word,type and definition alphabetical

In [ ]:
import re
# Fix supertabular and add \textit to type
def fix_latex_line(line):
    if re.match(r"^\\begin{supertabular}", line):
        # Add column_format to supertabular}
        return '\\begin{supertabular}'+'{'+column_format+'}'
    if re.match(r"^\\.*{tabular}", line):
        # Remove {tabular}
        return ''
    if re.match(r"^\w+\s.*\(\w+\s?\w+?\)", line):
        # Italics
        return re.sub(r"(^\w+\s.*)(\(\w+\s?\w+?\))", r"\1\\textit{\2}", line)
    return line

In [ ]:
# 2 Column word + definition
data = load_data()
data = data[["word", "definition"]]

style = data.style.format(
    escape="latex",
    )
style = style.hide(axis='index')
style = style.hide(axis='columns')

column_format = 'p{1.2in}p{2.3in}p{1.2in}p{2.3in}'
latex = style.to_latex(
    environment='supertabular',
    column_format=column_format
)

latex_lines = latex.splitlines()
latex = '\n'.join((map(fix_latex_line, latex_lines)))

filename = DATASET + '_table_alphabetical'
with open(f'./build/{filename}.tex', 'w') as f:
    f.write(latex)

/tmp/ipykernel_119920/3163655802.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


## 2 Column LateX word,type and definition by CEFR

In [ ]:
data = load_data()
cefrs = ['A1', 'A2', 'B1', 'B2', 'C1']
data_by_cefr = list(map(lambda c : data[data['cefr'] == c], cefrs))

/tmp/ipykernel_119920/3163655802.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cefr'] = data['cefr'].map(lambda x: x.strip().upper())


In [ ]:
data_by_cefr[1].head()

,word,type,cefr,phonetics (UK),phonetics (US),definition,example


In [ ]:
for data, cefr in zip(data_by_cefr, cefrs):
    if data.empty:
        continue
    data = data[["word", "definition", "type", "cefr"]]
    data["word"] = data.apply(lambda row: f"{row.word.strip()} ({row.type.strip()})" , axis=1)
    data = data[["word", "definition"]]

    style = data.style.format(
        escape="latex",
        )
    style = style.hide(axis='index')
    style = style.hide(axis='columns')

    column_format = 'p{1.2in}p{2.3in}p{1.2in}p{2.3in}'
    latex = style.to_latex(
        environment='supertabular',
        column_format=column_format
    )

    latex_lines = latex.splitlines()

    latex = '\n'.join((map(fix_latex_line, latex_lines)))

    filename = f'{DATASET}_{cefr}'
    with open(f'build/{filename}.tex', 'w') as f:
        f.write(latex)

/tmp/ipykernel_119920/1060421477.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["word"] = data.apply(lambda row: f"{row.word.strip()} ({row.type.strip()})" , axis=1)
/tmp/ipykernel_119920/1060421477.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["word"] = data.apply(lambda row: f"{row.word.strip()} ({row.type.strip()})" , axis=1)


## 2 Column LateX word,type and definition by CEFR shuffle

In [ ]:
for data, cefr in zip(data_by_cefr, cefrs):
    if data.empty:
        continue
    data = data[["word", "definition", "type", "cefr"]]
    data["word"] = data.apply(lambda row: f"{row.word.strip()} ({row.type.strip()})" , axis=1)
    data = data[["word", "definition"]]

    data = data.sample(frac = 1)

    style = data.style.format(
        escape="latex",
        )
    style = style.hide(axis='index')
    style = style.hide(axis='columns')

    column_format = 'p{1.2in}p{2.3in}p{1.2in}p{2.3in}'
    latex = style.to_latex(
        environment='supertabular',
        column_format=column_format
    )

    latex_lines = latex.splitlines()
    latex = '\n'.join((map(fix_latex_line, latex_lines)))

    filename = f'{DATASET}_shuffle_{cefr}'
    with open(f'build/{filename}.tex', 'w') as f:
        f.write(latex)

/tmp/ipykernel_119920/3056780930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["word"] = data.apply(lambda row: f"{row.word.strip()} ({row.type.strip()})" , axis=1)
/tmp/ipykernel_119920/3056780930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["word"] = data.apply(lambda row: f"{row.word.strip()} ({row.type.strip()})" , axis=1)


In [ ]:
# Run terminal cd format latexmk oxford*.tex to finish build pdfs
cmd_build = f"latexmk -pdf -cd format/{DATASET}*.tex -outdir=../output" 
os.system(cmd_build)

Rc files read:
  /etc/LatexMk
Latexmk: This is Latexmk, John Collins, 20 November 2021, version: 4.76.
Latexmk: Changing directory to 'format/'
Rule 'pdflatex': File changes, etc:
   Changed files, or newly in use since previous run(s):
      'oxford_5000_exclusive_two_column_alphabetical.tex'
------------
Run number 1 of rule 'pdflatex'
------------
------------
Running 'pdflatex  -recorder -output-directory="/home/jelle/wd/words/english/output"  "oxford_5000_exclusive_two_column_alphabetical.tex"'
------------


Latexmk: applying rule 'pdflatex'...
This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./oxford_5000_exclusive_two_column_alphabetical.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/generic/iftex/ifvtex.sty
(/usr/share/texlive/texmf-dist/tex/generic/iftex/iftex.sty)))
(/usr/share/texlive/texmf-dist/tex/latex/blindtext/blindtext.sty
(/usr/share/texlive/texmf-dist/tex/latex/tools/xspace.sty))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty
(/usr/share/texlive/texmf-dist/tex/latex/ucs/utf8x.def))
(/usr/sha

Latexmk: Log file says output to '/home/jelle/wd/words/english/output/oxford_5000_exclusive_two_column_alphabetical.pdf'
Rule 'pdflatex': File changes, etc:
   Changed files, or newly in use since previous run(s):
      '/home/jelle/wd/words/english/output/oxford_5000_exclusive_two_column_alphabetical.aux'
------------
Run number 2 of rule 'pdflatex'
------------
------------
Running 'pdflatex  -recorder -output-directory="/home/jelle/wd/words/english/output"  "oxford_5000_exclusive_two_column_alphabetical.tex"'
------------


Latexmk: Examining '/home/jelle/wd/words/english/output/oxford_5000_exclusive_two_column_alphabetical.log'
=== TeX engine is 'pdfTeX'
Latexmk: applying rule 'pdflatex'...
This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./oxford_5000_exclusive_two_column_alphabetical.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/generic/iftex/ifvtex.sty
(/usr/share/texlive/texmf-dist/tex/generic/iftex/iftex.sty)))
(/usr/share/texlive/texmf-dist/tex/latex/blindtext/blindtext.sty
(/usr/share/texlive/texmf-dist/tex/latex/tools/xspa

Latexmk: Log file says output to '/home/jelle/wd/words/english/output/oxford_5000_exclusive_two_column_alphabetical.pdf'
Latexmk: Undoing directory change
Latexmk: Changing directory to 'format/'
Rule 'pdflatex': File changes, etc:
   Changed files, or newly in use since previous run(s):
      'oxford_5000_exclusive_two_column_by_cefr.tex'
------------
Run number 1 of rule 'pdflatex'
------------
------------
Running 'pdflatex  -recorder -output-directory="/home/jelle/wd/words/english/output"  "oxford_5000_exclusive_two_column_by_cefr.tex"'
------------


...

Latexmk: Examining '/home/jelle/wd/words/english/output/oxford_5000_exclusive_two_column_by_cefr.log'
=== TeX engine is 'pdfTeX'
Latexmk: All targets (/home/jelle/wd/words/english/output/oxford_5000_exclusive_two_column_by_cefr.pdf) are up-to-date
Latexmk: applying rule 'pdflatex'...
This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./oxford_5000_exclusive_two_column_by_cefr_shuffle.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/generic/iftex/ifvtex.sty
(/usr/share/texlive/texmf-dist/tex/generic/iftex/iftex.sty)))

Latexmk: Missing input file 'oxford_5000_exclusive_two_column_by_cefr_shuffle.toc' (or dependence on it) from following:
  'No file oxford_5000_exclusive_two_column_by_cefr_shuffle.toc.'
Latexmk: Log file says output to '/home/jelle/wd/words/english/output/oxford_5000_exclusive_two_column_by_cefr_shuffle.pdf'
Rule 'pdflatex': File changes, etc:
   Changed files, or newly in use since previous run(s):
      '/home/jelle/wd/words/english/output/oxford_5000_exclusive_two_column_by_cefr_shuffle.aux'
      '/home/jelle/wd/words/english/output/oxford_5000_exclusive_two_column_by_cefr_shuffle.toc'
------------
Run number 2 of rule 'pdflatex'
------------
------------
Running 'pdflatex  -recorder -output-directory="/home/jelle/wd/words/english/output"  "oxford_5000_exclusive_two_column_by_cefr_shuffle.tex"'
------------



Underfull \hbox (badness 10000) in paragraph at lines 321--321
[]\OT1/phv/m/n/10 homeless \OT1/phv/m/sl/10 (ad-jec-

Underfull \hbox (badness 1742) in paragraph at lines 323--324
[]\OT1/phv/m/n/10 the work of col-lect-ing and writ-ing

Underfull \hbox (badness 1454) in paragraph at lines 326--327
[]\OT1/phv/m/n/10 a greater in-ter-est in or de-sire for

Underfull \hbox (badness 10000) in paragraph at lines 326--327
\OT1/phv/m/n/10 some-body/something than some-

Underfull \hbox (badness 1521) in paragraph at lines 344--345
[]\OT1/phv/m/n/10 a per-son who be-lieves that some

Overfull \hbox (11.64378pt too wide) in paragraph at lines 321--345
[] 

Underfull \hbox (badness 10000) in paragraph at lines 345--345
[]\OT1/phv/m/n/10 conservation

Underfull \hbox (badness 3039) in paragraph at lines 346--347
\OT1/phv/m/n/10 tances, es-pe-cially one con-nect-ing

Underfull \hbox (badness 1418) in paragraph at lines 348--349
[]\OT1/phv/m/n/10 the fact of be-ing male or fe-male,

Underfull \hbox

Latexmk: Log file says output to '/home/jelle/wd/words/english/output/oxford_5000_exclusive_two_column_by_cefr_shuffle.pdf'
Latexmk: Undoing directory change


0

In [ ]:
# Clean any build files in output/
files_to_remove = ["*.fls", "*.log", "*.html", "*.toc", "*.synctex*", "*.fdb*", "*.aux"]
cmd_cleanup_output = "rm "+" ".join(f"output/{f}" for f in files_to_remove)
cmd_cleanup_format = "rm "+" ".join(f"format/{f}" for f in files_to_remove+["*.pdf"])
cmd_cleanup_main = "rm "+" ".join(f"./{f}" for f in files_to_remove)
os.system(";".join((cmd_cleanup_format, cmd_cleanup_main, cmd_cleanup_output)))

rm: cannot remove 'format/*.fls': No such file or directory
rm: cannot remove 'format/*.log': No such file or directory
rm: cannot remove 'format/*.html': No such file or directory
rm: cannot remove 'format/*.toc': No such file or directory
rm: cannot remove 'format/*.synctex*': No such file or directory
rm: cannot remove 'format/*.fdb*': No such file or directory
rm: cannot remove 'format/*.aux': No such file or directory
rm: cannot remove 'format/*.pdf': No such file or directory
rm: cannot remove './*.fls': No such file or directory
rm: cannot remove './*.log': No such file or directory
rm: cannot remove './*.html': No such file or directory
rm: cannot remove './*.toc': No such file or directory
rm: cannot remove './*.synctex*': No such file or directory
rm: cannot remove './*.fdb*': No such file or directory
rm: cannot remove './*.aux': No such file or directory
rm: cannot remove 'output/*.synctex*': No such file or directory


256